In [1]:
import os
import torch
import requests
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
from langchain.prompts import ChatPromptTemplate


In [2]:
def config():
    load_dotenv()

config()

In [3]:
API_URL = "https://api-inference.huggingface.co/models/google/gemma-7b-it"
headers = {"Authorization": "Bearer {}".format(os.getenv('API_KEY'))}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)    
    return response.json()


In [4]:
# To check input given to model.
def gemma_template(prompt):
    text =  "<start_of_turn>user\n{}<end_of_turn>".format(prompt) + "\n<start_of_turn>model"    
    return text

In [5]:
def output(prompt):
    text =  gemma_template(prompt)   
    
    return query({
        "inputs":text,
        "parameters":{"top_k":2,
                     "top_p":0.9,
                     "temperature":0.5,
                     "max_length":500}  
        
    })[0]['generated_text'][len(text):].strip()



In [ ]:
def translate_customer_message(template_string,customer_style,customer_email):
    chat_pt = ChatPromptTemplate.from_template(template_string)
    customer_message = chat_pt.format_messages(style=customer_style,text=customer_email)[0].content
    print(output(customer_message))
    



In [6]:
template_string = """Translate the text that is delimited by triple backticks into a style that is {style}.\n 
text: ```{text}```
"""

In [7]:
chat_pt = ChatPromptTemplate.from_template(template_string)
chat_pt

ChatPromptTemplate(input_variables=['style', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}.\n \ntext: ```{text}```\n'))])

In [8]:
customer_style = """American English 
in a calm and respectful tone
""".strip()

customer_email = """Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!""".strip()

In [9]:
customer_message = chat_pt.format_messages(style=customer_style,text=customer_email)[0].content
print(customer_message)

Translate the text that is delimited by triple backticks into a style that is American English 
in a calm and respectful tone.
 
text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```



In [11]:
print(gemma_template(customer_message))

<start_of_turn>user
Translate the text that is delimited by triple backticks into a style that is American English 
in a calm and respectful tone.
 
text: ```Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!```
<end_of_turn>
<start_of_turn>model


In [10]:
print(output(customer_message))

**American English Translation:**

"Hey, listen up, my friend. I'm pretty upset about what happened earlier. My blender lid flew off and plastered my kitchen walls with a smoothie. To top it all off, the warranty doesn't cover the cost of cleaning up the mess. I'm in a real bind, so I need your help right now, my good man."
